89% accuracy with data agumentation on https://www.kaggle.com/datasets/mohammadhossein77/brain-tumors-dataset

In [1]:
!pip install opendatasets
!pip install pandas

In [2]:
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/mohammadhossein77/brain-tumors-dataset")

100%|██████████| 199M/199M [00:09<00:00, 20.9MB/s]


In [4]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [5]:
dataset_path = '/content/brain-tumors-dataset/Data'
normal_folder = os.path.join(dataset_path, 'Normal')
tumor_folder = os.path.join(dataset_path, 'Tumor')
pituitary_folder = os.path.join(tumor_folder, 'pituitary_tumor')
glioma_folder = os.path.join(tumor_folder, 'glioma_tumor')
meningioma_folder = os.path.join(tumor_folder, 'meningioma_tumor')


In [6]:
train_path = '/content/train'
test_path = '/content/test'

os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)


In [7]:
normal_files = os.listdir(normal_folder)
normal_train, normal_test = train_test_split(normal_files, test_size=0.2, random_state=42)
os.makedirs(os.path.join(train_path, 'Normal'), exist_ok=True)
os.makedirs(os.path.join(test_path, 'Normal'), exist_ok=True)

for file_name in normal_train:
    src = os.path.join(normal_folder, file_name)
    dst = os.path.join(train_path, 'Normal', file_name)
    shutil.copy(src, dst)

for file_name in normal_test:
    src = os.path.join(normal_folder, file_name)
    dst = os.path.join(test_path, 'Normal', file_name)
    shutil.copy(src, dst)


In [8]:
def split_tumor_folder(tumor_folder, train_path, test_path):
    tumor_files = os.listdir(tumor_folder)
    tumor_train, tumor_test = train_test_split(tumor_files, test_size=0.2, random_state=42)

    tumor_train_path = os.path.join(train_path, os.path.basename(tumor_folder))
    tumor_test_path = os.path.join(test_path, os.path.basename(tumor_folder))
    os.makedirs(tumor_train_path, exist_ok=True)
    os.makedirs(tumor_test_path, exist_ok=True)

    for file_name in tumor_train:
        src = os.path.join(tumor_folder, file_name)
        dst = os.path.join(tumor_train_path, file_name)
        shutil.copy(src, dst)

    for file_name in tumor_test:
        src = os.path.join(tumor_folder, file_name)
        dst = os.path.join(tumor_test_path, file_name)
        shutil.copy(src, dst)

split_tumor_folder(pituitary_folder, train_path, test_path)
split_tumor_folder(glioma_folder, train_path, test_path)
split_tumor_folder(meningioma_folder, train_path, test_path)


In [29]:
import tensorflow as tf
from keras import layers


model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


# Prepare the dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    image_size=(224,224),
    batch_size=32
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    image_size=(224,224),
    batch_size=32
)


Found 15264 files belonging to 4 classes.
Found 3819 files belonging to 4 classes.


In [30]:
model.fit(train_ds, validation_data=test_ds, epochs=4)

Epoch 1/4
477/477 [==============================] - 31s 62ms/step - loss: -2446852620288.0000 - accuracy: 0.3305 - val_loss: -13246124785664.0000 - val_accuracy: 0.3305
Epoch 2/4
477/477 [==============================] - 30s 62ms/step - loss: -85782920429568.0000 - accuracy: 0.3305 - val_loss: -220409249136640.0000 - val_accuracy: 0.3305
Epoch 3/4
477/477 [==============================] - 28s 58ms/step - loss: -549985359757312.0000 - accuracy: 0.3305 - val_loss: -1016889476120576.0000 - val_accuracy: 0.3305
Epoch 4/4
477/477 [==============================] - 28s 58ms/step - loss: -1832460010651648.0000 - accuracy: 0.3305 - val_loss: -2863713199587328.0000 - val_accuracy: 0.3305


In [31]:
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

120/120 [==============================] - 12s 98ms/step - loss: 10094216478720.0000 - accuracy: 0.2500
Test Loss: 10094216478720.0
Test Accuracy: 0.25


In [32]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')  # Changed the number of units to match the number of classes
])


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Prepare the dataset with data augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=15,  # Randomly rotate images
    width_shift_range=0.1,  # Randomly shift images horizontally
    height_shift_range=0.1,  # Randomly shift images vertically
    shear_range=0.1,  # Apply shear transformation
    zoom_range=0.1,  # Apply zoom transformation
    horizontal_flip=True  # Randomly flip images horizontally
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Set class_mode to 'categorical' for multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Set class_mode to 'categorical' for multi-class classification
)

Found 15264 images belonging to 4 classes.
Found 3819 images belonging to 4 classes.


In [33]:
model.fit(train_generator, validation_data=test_generator, epochs=20)

Epoch 1/20
477/477 [==============================] - 204s 423ms/step - loss: 0.9691 - accuracy: 0.5688 - val_loss: 0.8388 - val_accuracy: 0.6444
Epoch 2/20
477/477 [==============================] - 201s 422ms/step - loss: 0.7989 - accuracy: 0.6547 - val_loss: 0.6628 - val_accuracy: 0.7340
Epoch 3/20
477/477 [==============================] - 201s 422ms/step - loss: 0.6866 - accuracy: 0.7148 - val_loss: 0.5816 - val_accuracy: 0.7690
Epoch 4/20
477/477 [==============================] - 201s 421ms/step - loss: 0.5925 - accuracy: 0.7615 - val_loss: 0.4907 - val_accuracy: 0.8104
Epoch 5/20
477/477 [==============================] - 203s 425ms/step - loss: 0.5157 - accuracy: 0.7976 - val_loss: 0.4719 - val_accuracy: 0.8128
Epoch 6/20
477/477 [==============================] - 201s 422ms/step - loss: 0.4687 - accuracy: 0.8141 - val_loss: 0.4639 - val_accuracy: 0.8230
Epoch 7/20
477/477 [==============================] - 202s 424ms/step - loss: 0.4215 - accuracy: 0.8346 - val_loss: 0.4407 -

In [45]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

120/120 [==============================] - 9s 77ms/step - loss: 0.2751 - accuracy: 0.8960
Test Loss: 0.2750990092754364
Test Accuracy: 0.8960461020469666
